In [3]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from ipywidgets import FloatSlider, VBox, HBox, interactive_output

np.random.seed(42)
n = 400

# --------------------------
# Data generator
# --------------------------
def generate_data(var_x=5.0, var_y=2.0, var_z=1.0,
                  rot_x=0, rot_y=0, rot_z=0):
    """Generate rotated 3D Gaussian ellipsoid data."""
    u = np.sqrt(var_x) * np.random.randn(n)
    v = np.sqrt(var_y) * np.random.randn(n)
    w = np.sqrt(var_z) * np.random.randn(n)
    data = np.vstack([u, v, w]).T

    Rx = np.array([[1, 0, 0],
                   [0, np.cos(np.deg2rad(rot_x)), -np.sin(np.deg2rad(rot_x))],
                   [0, np.sin(np.deg2rad(rot_x)),  np.cos(np.deg2rad(rot_x))]])

    Ry = np.array([[np.cos(np.deg2rad(rot_y)), 0, np.sin(np.deg2rad(rot_y))],
                   [0, 1, 0],
                   [-np.sin(np.deg2rad(rot_y)), 0, np.cos(np.deg2rad(rot_y))]])

    Rz = np.array([[np.cos(np.deg2rad(rot_z)), -np.sin(np.deg2rad(rot_z)), 0],
                   [np.sin(np.deg2rad(rot_z)),  np.cos(np.deg2rad(rot_z)), 0],
                   [0, 0, 1]])

    R = Rz @ Ry @ Rx
    return data @ R.T

# --------------------------
# Euler projection planes
# --------------------------
def project_to_plane(data, axis1, axis2):
    """Project data onto a 2D plane spanned by axis1, axis2."""
    coords1 = data @ axis1
    coords2 = data @ axis2
    return coords1, coords2

def euler_axes(alpha, beta, gamma):
    """Return three orthogonal axes based on Euler rotations."""
    Rx = np.array([[1, 0, 0],
                   [0, np.cos(np.deg2rad(alpha)), -np.sin(np.deg2rad(alpha))],
                   [0, np.sin(np.deg2rad(alpha)),  np.cos(np.deg2rad(alpha))]])

    Ry = np.array([[np.cos(np.deg2rad(beta)), 0, np.sin(np.deg2rad(beta))],
                   [0, 1, 0],
                   [-np.sin(np.deg2rad(beta)), 0, np.cos(np.deg2rad(beta))]])

    Rz = np.array([[np.cos(np.deg2rad(gamma)), -np.sin(np.deg2rad(gamma)), 0],
                   [np.sin(np.deg2rad(gamma)),  np.cos(np.deg2rad(gamma)), 0],
                   [0, 0, 1]])

    R = Rz @ Ry @ Rx
    return R[:,0], R[:,1], R[:,2]  # three orthogonal axes

# --------------------------
# Plot 1: 3D interactive
# --------------------------
def plot_3d(var_x=5.0, var_y=2.0, var_z=1.0, rot_x=0, rot_y=0, rot_z=0):
    data = generate_data(var_x, var_y, var_z, rot_x, rot_y, rot_z)
    X, Y, Z = data[:,0], data[:,1], data[:,2]
    L = np.max(np.abs(data)) * 1.2

    fig = go.Figure(data=[go.Scatter3d(
        x=X, y=Y, z=Z, mode='markers',
        marker=dict(size=3, color='blue', opacity=0.6)
    )])
    fig.update_layout(scene=dict(
        xaxis=dict(range=[-L, L]),
        yaxis=dict(range=[-L, L]),
        zaxis=dict(range=[-L, L]),
        aspectmode='cube'
    ),
    title="Interactive 3D Scatter (Plot 1)")
    fig.show()

# --------------------------
# Plot 2: 2×3 projections
# --------------------------
def plot_2d(var_x=5.0, var_y=2.0, var_z=1.0,
            rot_x=0, rot_y=0, rot_z=0,
            alpha=0, beta=0, gamma=0):
    data = generate_data(var_x, var_y, var_z, rot_x, rot_y, rot_z)
    X, Y, Z = data[:,0], data[:,1], data[:,2]
    L = np.max(np.abs(data)) * 1.2

    # Euler axes
    ax_alpha, ax_beta, ax_gamma = euler_axes(alpha, beta, gamma)

    fig, axes = plt.subplots(2, 3, figsize=(14,8))

    # --- Top row: XY, YZ, ZX ---
    axes[0,0].scatter(X, Y, alpha=0.5, c="blue", s=10)
    axes[0,0].set_xlim(-L, L); axes[0,0].set_ylim(-L, L)
    axes[0,0].set_aspect("equal"); axes[0,0].set_title("XY projection")

    axes[0,1].scatter(Y, Z, alpha=0.5, c="blue", s=10)
    axes[0,1].set_xlim(-L, L); axes[0,1].set_ylim(-L, L)
    axes[0,1].set_aspect("equal"); axes[0,1].set_title("YZ projection")

    axes[0,2].scatter(Z, X, alpha=0.5, c="blue", s=10)
    axes[0,2].set_xlim(-L, L); axes[0,2].set_ylim(-L, L)
    axes[0,2].set_aspect("equal"); axes[0,2].set_title("ZX projection")

    # --- Bottom row: αβ, βγ, γα ---
    u,v = project_to_plane(data, ax_alpha, ax_beta)
    axes[1,0].scatter(u, v, alpha=0.5, c="red", s=10)
    axes[1,0].set_xlim(-L, L); axes[1,0].set_ylim(-L, L)
    axes[1,0].set_aspect("equal"); axes[1,0].set_title("α–β projection")

    u,v = project_to_plane(data, ax_beta, ax_gamma)
    axes[1,1].scatter(u, v, alpha=0.5, c="red", s=10)
    axes[1,1].set_xlim(-L, L); axes[1,1].set_ylim(-L, L)
    axes[1,1].set_aspect("equal"); axes[1,1].set_title("β–γ projection")

    u,v = project_to_plane(data, ax_gamma, ax_alpha)
    axes[1,2].scatter(u, v, alpha=0.5, c="red", s=10)
    axes[1,2].set_xlim(-L, L); axes[1,2].set_ylim(-L, L)
    axes[1,2].set_aspect("equal"); axes[1,2].set_title("γ–α projection")

    plt.tight_layout()
    plt.show()

# --------------------------
# Widgets
# --------------------------
varx_slider = FloatSlider(min=0.1, max=10, step=0.1, value=5, description="Var X")
vary_slider = FloatSlider(min=0.1, max=10, step=0.1, value=2, description="Var Y")
varz_slider = FloatSlider(min=0.1, max=10, step=0.1, value=1, description="Var Z")

rotx_slider = FloatSlider(min=0, max=180, step=5, value=20, description="Rot X")
roty_slider = FloatSlider(min=0, max=180, step=5, value=10, description="Rot Y")
rotz_slider = FloatSlider(min=0, max=180, step=5, value=0, description="Rot Z")

alpha_slider = FloatSlider(min=0, max=180, step=5, value=0, description="α")
beta_slider = FloatSlider(min=0, max=180, step=5, value=0, description="β")
gamma_slider = FloatSlider(min=0, max=180, step=5, value=0, description="γ")

# Interactive outputs
out1 = interactive_output(plot_3d, {
    "var_x": varx_slider, "var_y": vary_slider, "var_z": varz_slider,
    "rot_x": rotx_slider, "rot_y": roty_slider, "rot_z": rotz_slider
})

out2 = interactive_output(plot_2d, {
    "var_x": varx_slider, "var_y": vary_slider, "var_z": varz_slider,
    "rot_x": rotx_slider, "rot_y": roty_slider, "rot_z": rotz_slider,
    "alpha": alpha_slider, "beta": beta_slider, "gamma": gamma_slider
})

ui = VBox([
    HBox([varx_slider, vary_slider, varz_slider]),
    HBox([rotx_slider, roty_slider, rotz_slider]),
    out1,   # Plotly interactive 3D
    HBox([alpha_slider, beta_slider, gamma_slider]),
    out2    # Matplotlib projections
])
ui
